In [1]:
#clip and faiss image similarity search
import os
import pandas as pd
import numpy as np
from PIL import Image
from torchvision import models, transforms
import faiss
import torch
from sentence_transformers import SentenceTransformer

device = torch.device("cpu")

data = [
        ['0029_RV.png'],
        ['0080_MVL.png'],
        ['0105_MVL.png']
       ]

# code if images are taken from a folder
# source_path = r'<source_path>'
# data = [f for f in os.listdir(source_path) if f.endswith((".jpg", ".jpeg", ".png"))]
# for i in range(len(data)):
#     # Construct the full path to the image file
#     data[i] = os.path.join(source_path, data[i])

df = pd.DataFrame(data, columns=['image_path'])
df.head()
model = SentenceTransformer('clip-ViT-B-32')

def get_image_embedding(input_image):
    with torch.no_grad():
        embedding = model.encode(Image.open(input_image))
        embedding.shape
    return embedding

embeddings = np.vstack([get_image_embedding(path) for path in df['image_path']])
print(embeddings.shape)
vector_dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(vector_dimension)
index.add(embeddings)

search_image_path = '0154_RV.png'
search_vector = get_image_embedding(search_image_path)

search_vector = np.array([search_vector.squeeze()])

print("Shape of search_vector:", search_vector.shape)

k = index.ntotal
D, I = index.search(search_vector, k=k)  # Perform the search

print(D, I)

top_indices = I[0]
top_results = df.iloc[top_indices]

top_results['distance'] = D[0]

print(top_results.head(1))

C:\Users\gssabijp\AppData\Roaming\Python\Python311\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()



(3, 512)
Shape of search_vector: (1, 512)
[[14.925858 51.629223 88.356514]] [[0 2 1]]
    image_path   distance
0  0029_RV.png  14.925858


C:\Users\gssabijp\AppData\Local\Temp\ipykernel_22800\2113816697.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_results['distance'] = D[0]
